In [6]:
import json

In [45]:
def load_dataset(dataset_path):
    """
    加载测试集，并计算其中的 tokens 总数。

    :param dataset_path: 测试集的文件路径。
    :return: 测试数据集及其 tokens 总数。
    """
    with open(dataset_path, 'r', encoding='utf-8') as file:
        dataset = [json.loads(line) for line in file.readlines()]
    
    # 计算 tokens 总数
    total_tokens = sum(len(item['messages'][0]['content']) + len(item['messages'][1]['content']) for item in dataset)*8

    return dataset, total_tokens


In [63]:
def memory_usage(model_parameters, batch_size, data_type_size=1, overhead_factor=1.5, num_gpus=1):
    """
    估算训练过程中的显存使用量（以GB为单位）。

    :param model_parameters: 模型的参数总量。
    :param batch_size: 训练过程中的batch大小。
    :param data_type_size: 数据类型的大小（以字节为单位），对于32位浮点数通常为4。
    :param overhead_factor: 额外开销因子。
    :param num_gpus: 使用的 GPU 数量。
    :return: 显存使用量（GB）。
    """
    return (model_parameters * data_type_size / 5e10) * (1 + overhead_factor) * batch_size / num_gpus

def evaluation_duration(num_tokens, model_parameters, gpu_flops, gpu_utilization=0.3, num_gpus=1):
    """
    估算整个评估过程的持续时间（以天为单位）。

    :param num_tokens: 训练过程中使用的tokens总数。
    :param model_parameters: 模型的参数总量。
    :param gpu_flops: GPU的峰值性能。
    :param gpu_utilization: GPU利用率。
    :param num_gpus: 使用的 GPU 数量。
    :return: 评估持续时间（天）。
    """
    seconds_per_day = 86400
    flops_per_token = 8
    return (num_tokens * model_parameters * flops_per_token) / (gpu_flops * gpu_utilization * seconds_per_day * num_gpus)

In [64]:
# 加载数据集并计算 tokens 总数
dataset_path = "../mnt/datasets/test.jsonl"
dataset, total_tokens = load_dataset(dataset_path)
print(f"测试集中的tokens总数：{total_tokens}")

# 设置模型和训练参数
model_parameters = 130e9  # 130亿参数
batch_size = 8
gpu_flops = 624e12  # 312 TFLOPS
num_gpus = 1

# 计算显存使用和评估持续时间
memory = memory_usage(model_parameters, batch_size, num_gpus=num_gpus)
duration = evaluation_duration(total_tokens, model_parameters, gpu_flops, num_gpus=num_gpus)

print(f"预估显存使用量：{memory} GB per GPU")
# duration转为分钟
print(f"预估评估持续时间：{duration:.2f} 天，即 {duration * 24 * 60:.2f} 分钟")

测试集中的tokens总数：209712
预估显存使用量：52.0 GB per GPU
预估评估持续时间：0.01 天，即 19.42 分钟
